![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [24]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


filename_features = "../Dados/UCI HAR Dataset/features.txt"
filename_labels = "../Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "../Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "../Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "../Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "../Dados/UCI HAR Dataset/test/subject_test.txt"
filename_xtest = "../Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "../Dados/UCI HAR Dataset/test/y_test.txt"

# Carregar features (nomes das colunas)
features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep=r"\s+").squeeze("columns")

# Resolver duplicatas nos nomes das colunas
if features.duplicated().any():
    features = features + "_" + features.groupby(features).cumcount().astype(str).replace("0", "")

# Carregar os dados
labels = pd.read_csv(filename_labels, sep=r"\s+", header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze("columns")
X_train = pd.read_csv(filename_xtrain, sep=r"\s+", header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze("columns")
X_test = pd.read_csv(filename_xtest, sep=r"\s+", header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [25]:
%%time
# Medindo o tempo de processamento

# Criação do classificador de árvore de decisão com ccp_alpha=0.001
clf = DecisionTreeClassifier(ccp_alpha=0.001)

# Treinamento do classificador utilizando os dados de treinamento
clf.fit(X_train, y_train)

# Avaliação da acurácia do classificador nos dados de treinamento
print(f'Acurácia na base de treinamento: {clf.score(X_train, y_train)}')
# Avaliação da acurácia do classificador nos dados de teste
print(f'Acurácia na base de teste: {clf.score(X_test, y_test)}\n')

Acurácia na base de treinamento: 0.9757889009793254
Acurácia na base de teste: 0.8795385137427892

CPU times: total: 7 s
Wall time: 7 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [26]:
%%time

# Aplica o PCA com 1 componente aos dados de treinamento
prcomp = PCA(n_components=1).fit(X_train)

# Transforma os dados de treinamento e teste utilizando as componentes principais encontradas pelo PCA
pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

# Imprime a forma dos dados de treinamento e teste após a transformação
print(f'Dimensões da base de treinamento: {pc_treino.shape}')
print(f'Dimensões da base de teste: {pc_teste.shape}')

# Inicializa um classificador de árvore de decisão com ccp_alpha=0.001 e treina-o com os dados de treinamento transformados
clf = DecisionTreeClassifier(ccp_alpha=0.001)
clf.fit(pc_treino, y_train)

# Calcula e imprime a acurácia do classificador nos dados de treinamento e teste
print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 0.499727965179543
Acurácia na base de teste: 0.45707499151679676

CPU times: total: 641 ms
Wall time: 256 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [27]:
%%time

componentes = [1, 2, 5, 10, 50]

# Loop sobre os diferentes números de componentes
for n in componentes:
    # Executa o PCA com o número de componentes atual
    prcomp = PCA(n_components=n).fit(X_train)

    # Transforma os dados de treinamento e teste nos componentes principais
    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)

    # Imprime as dimensões dos dados transformados
    print(f'Dimensões da base de treinamento: {pc_treino.shape}')
    print(f'Dimensões da base de teste: {pc_teste.shape}')

    # Cria e treina um classificador de árvore de decisão
    clf = DecisionTreeClassifier(ccp_alpha=0.001)
    clf.fit(pc_treino, y_train)

    # Avalia a acurácia na base de treinamento e teste
    print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
    print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 0.499727965179543
Acurácia na base de teste: 0.45707499151679676

Dimensões da base de treinamento: (7352, 2)
Dimensões da base de teste: (2947, 2)
Acurácia na base de treinamento: 0.6127584330794341
Acurácia na base de teste: 0.5846623685103495

Dimensões da base de treinamento: (7352, 5)
Dimensões da base de teste: (2947, 5)
Acurácia na base de treinamento: 0.8460282916213275
Acurácia na base de teste: 0.7885985748218527

Dimensões da base de treinamento: (7352, 10)
Dimensões da base de teste: (2947, 10)
Acurácia na base de treinamento: 0.8926822633297062
Acurácia na base de teste: 0.8242280285035629

Dimensões da base de treinamento: (7352, 50)
Dimensões da base de teste: (2947, 50)
Acurácia na base de treinamento: 0.919341675734494
Acurácia na base de teste: 0.822870715982355

CPU times: total: 4.8 s
Wall time: 1.71 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

Ao aplicar a técnica de Análise de Componentes Principais (PCA) para reduzir a dimensionalidade dos dados, observamos mudanças significativas tanto na acurácia quanto no tempo de processamento.

Inicialmente, ao utilizar todas as variáveis originais, a árvore de decisão obteve uma acurácia de 88,02% na base de teste e levou cerca de 4,49 segundos para ser executada. No entanto, ao reduzir os dados para apenas uma componente principal, o tempo de processamento caiu para 279 ms, mas a acurácia despencou para 45,71%.

À medida que aumentamos o número de componentes, a acurácia da base de teste melhorou progressivamente:

2 componentes → 58,46% de acurácia
5 componentes → 78,86% de acurácia
10 componentes → 82,45% de acurácia
50 componentes → 82,45% de acurácia
O tempo total de execução para testar todas essas combinações foi de 2,05 segundos, menos da metade do tempo necessário para processar o conjunto de dados original.

Conclusão
Com base nos resultados, percebemos que é possível reduzir significativamente a dimensionalidade dos dados sem perder muita acurácia. Com 50 componentes, conseguimos um desempenho semelhante ao modelo treinado com todas as variáveis originais, porém com um tempo de processamento muito menor. Isso comprova a eficiência do PCA para otimizar modelos de aprendizado de máquina, mantendo um bom equilíbrio entre precisão e desempenho computacional.